In [1]:
!unzip enron.zip -d data
#First upload enron.zip, then unzip it and restore it in data folder
#data folder path is /content/data/

In [4]:
import glob
import re
import hashlib
import networkx as nx
import time
import numpy
import operator
import math
import sys

#This function was copied from http://stackoverflow.com/questions/2545532/python-analog-of-natsort-function-sort-a-list-using-a-natural-order-algorithm
#It allows for natural sorting for the input files, so they're read in the correct order
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]

#Get command line args
graphFilePath = sys.argv[1:]
graphFilePath = str("/content/data")
#graphFilePath = ""
print ("Loading files from folder: " + graphFilePath)

start_time = time.clock()

#Load dataset
# files = ["datasets/enron/0_enron_by_day.txt", "datasets/enron/1_enron_by_day.txt", "datasets/enron/2_enron_by_day.txt",
# "datasets/enron/3_enron_by_day.txt", "datasets/enron/4_enron_by_day.txt"]  #
print (graphFilePath + "*.txt")
files = glob.glob(graphFilePath + "*.txt")
files = glob.glob("/content/data/*.txt")

#print(files)
sorted_files = sorted(files, key=natural_key)

# Create directed graph in networkx
list_All_Feature_Set = []

#Loop through each graph file
for file in sorted_files:
    f = open(file, "r")
    dataGraph = f.readlines()

    #Create networkx graph object
    G = nx.DiGraph()

    # Add edges to Graph object from input file
    for line in dataGraph:
        nodeA, nodeB = line.split(" ")
        # print(nodeA)
        # print(nodeB)
        nodeA.rstrip('\r\n')
        nodeB.rstrip('\r\n')
        nodeB = nodeB[:-1]
        G.add_edge(str(nodeA), str(nodeB))

    #Generate dictionary with pagerank (quality) values for each node
    weighted_Feature_Set = nx.pagerank(G)

    

    #Add edge "quality" from graph to weighted feature set
    for a, b in G.edges():
        weighted_Feature_Set[str(a) + " " + str(b)] = (1/len(G.edges(a))) * weighted_Feature_Set[a]
    #THIS NEEDS TO BE IMPROVED!!!

    print(len(weighted_Feature_Set)) 
    print(weighted_Feature_Set)

    #Convert every key to a md5 hash
    for key, value in weighted_Feature_Set.items():
        sha1_hash = hashlib.sha1(str(key).encode() + str(value).encode()).hexdigest()
        binary_hash = bin(int(sha1_hash, 16))[2:]
        #Trim to fixed size of 128
        binary_hash = (binary_hash[:128]) if len(binary_hash) > 128 else binary_hash
        #print(len(binary_hash))
        # print("jksdjfskdfjdksfj")
        # print(str(binary_hash))
        
        #Add sha1 hash with corresponding value to feature set
        weighted_Feature_Set[str(binary_hash)] = value
        # print("shfskfhksf")
        # print(weighted_Feature_Set)
        # #Remove original key/value from feature set
        del weighted_Feature_Set[key]

    #Add file
    
    print(len(weighted_Feature_Set))
    print(weighted_Feature_Set)
    list_All_Feature_Set.append(weighted_Feature_Set)

    print ("Processed file: " + file)


#List of all document fingerprints
all_Fingerprints = []

#Loop through each feature set to generate it's fingerprint
for feature_Set in list_All_Feature_Set:
    temp_fingerprint = []
    #Build fingerprint
    #print("dnskjksfjsd")
    # for key in feature_Set:
    #     print(key, ' : ', feature_Set[key])
    for key, val in feature_Set.items():
        binary_list = []
        #For each sha1 hash, if digit=1 then add weighted value. If digit=0, then add negative weighted value.
        #print("sjhfasjfhsaj")
        #print(key, ' : ', val)

        if( len(key) != 128):
          continue

        for d in key:
            if int(d) == 1:
                binary_list.append(val)
            else:
                binary_list.append(-val)
        temp_fingerprint.append(binary_list)

    # print("jfhsdjfhsdjhf")
    # print(temp_fingerprint)    

    #Calculate fingerprint by summing all the columns in temp_fingerprint
    fingerprint = [sum(x) for x in zip(*temp_fingerprint)]
  

    #Convert all positive vals to '1' and all negative vals to '0' in fingerprint
    for index, item in enumerate(fingerprint):
        if item > 0:
            fingerprint[index] = 1
        else:
            fingerprint[index] = 0

    #Add fingerprint to list
    #print("fingerint")
    
    all_Fingerprints.append(fingerprint)

    # print(all_Fingerprints)

    # print(len(all_Fingerprints[0]))

    # print("fingeint")


#Calculate similarity scores (hamming distance) between each sequential pair of fingerprints
similarity_scores = []
for x in range(len(all_Fingerprints) - 1):
    hamming = 0.0
    #Calculate hamming distance
    for index in range(0, 128):
        if all_Fingerprints[x][index] == all_Fingerprints[x + 1][index]:
            hamming += 1
    similarity_scores.append(1-(hamming/128))
    #print ("Similarity score between " + str(x) + " and " + str(x+1) + " : " + str(1-(hamming / 128)))

print(similarity_scores)

#Moving Range Average
median = numpy.median(numpy.array(similarity_scores))
thresholdList = []
tempAvgNumerator = 0
multiplierForMR = 1
for y in range(len(similarity_scores)):
    if y == 0:
        tempAvgNumerator = 0
        thresholdList.append(0)
    else:
        tempAvgNumerator = tempAvgNumerator + abs(similarity_scores[y]-similarity_scores[y-1])
        threshold = median - (multiplierForMR*tempAvgNumerator/y)
        thresholdList.append(threshold)

#Detect anomalies || Look for two consecutive below threshold
anomalies = {}
anomaliesscore = {}
score = []
for x in range(len(similarity_scores)-1):
    #Detect two consecutive anomalies
    if (similarity_scores[x] < thresholdList[x]) and (similarity_scores[x+1] < thresholdList[x+1]):
        anomalies[str(x+1)] = abs(similarity_scores[x] - thresholdList[x]) + abs(similarity_scores[x+1] - thresholdList[x+1])
        anomaliesscore[str(x+1)] = similarity_scores[x+1]

###Output Anomalies###
#You will list all of the anomalous time points if
#there are fewer than 10, the top 10 if there are fewer than 100, or the top 10% if there are
#more than 100.
f = open('anomalies_output', 'w')
sorted_anomalies = sorted(anomalies.items(), key=operator.itemgetter(1), reverse=True)
numOfAnomalies = len(sorted_anomalies)
if numOfAnomalies > 100:
    print ("There are " + str(numOfAnomalies) + " anomalies found, so we will output the top 10%")
    for x in range(0,int(math.ceil(0.1*len(sorted_anomalies)))):
        f.write(str(sorted_anomalies[x][0]))
        f.write("\n")
elif numOfAnomalies > 11:
    print ("There are " + str(numOfAnomalies) + " anomalies found, so we will output the top 10")
    for x in range(0,10):
        f.write(str(sorted_anomalies[x][0]))
        f.write("\n")
else:
    print ("There are " + str(numOfAnomalies) + " anomalies found, so we will output all of them")
    for x in sorted_anomalies:
        f.write(str(x[0]))
        f.write("\n")
f.close()

print ("Complete")
print("--- Total processing took %s seconds ---" % ( time.clock() - start_time))

##DEBUGGING AND GRAPHING OUTPUT
print ("Writing out to file: anomalies_output")
#Write out to file
f = open('similarity_scores', 'w')
for x in similarity_scores:
    f.write(str(x))
    f.write("\n")
f.close()

f = open('moving_thresholds', 'w')
for x in thresholdList:
    f.write(str(x))
    f.write("\n")
f.close()

Loading files from folder: /content/data
/content/data*.txt
6
{'184': 0.17543839772251535, '1': 0.32456160227748465, '81': 0.17543839772251535, '50': 0.32456160227748465, '184 1': 0.17543839772251535, '81 50': 0.17543839772251535}
6
{'81 50': 0.17543839772251535, '10001111110000100001000010011101100000011110101101011011110101010111001100110110100011101011101100110100111101110100101011101010': 0.17543839772251535, '10111101000101110001110001110000010101101101111101100001101011110101000001110110011010101000110101011001111110111011110100111100': 0.32456160227748465, '11001001011010000000010001000101000000101011001010011001011110000011111111101001011100001011110101011111011001101010011010110000': 0.17543839772251535, '10010101010101111010010001001000000010100100101110101110001000001111000011111000011001000101101111001111001101011101101000001011': 0.17543839772251535, '101011010001011001010111111011000011011000111000111010100100000010001011011101001000111101011010000010100010000011011001000

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[0.4140625, 0.484375, 0.4765625, 0.515625, 0.46875, 0.5234375, 0.5078125, 0.46875, 0.4765625, 0.5390625, 0.4765625, 0.4453125, 0.4765625, 0.4375, 0.5546875, 0.3515625, 0.484375, 0.546875, 0.515625, 0.5546875, 0.4296875, 0.5078125, 0.5234375, 0.5625, 0.4921875, 0.4375, 0.5390625, 0.5, 0.5390625, 0.5390625, 0.578125, 0.4453125, 0.53125, 0.5078125, 0.421875, 0.4765625, 0.46875, 0.4921875, 0.4609375, 0.3203125, 0.484375, 0.5390625, 0.484375, 0.453125, 0.5078125, 0.484375, 0.4296875, 0.4140625, 0.4921875, 0.46875, 0.5078125, 0.6015625, 0.4921875, 0.5859375, 0.4375, 0.5, 0.46875, 0.4765625, 0.4765625, 0.4765625, 0.515625, 0.53125, 0.4765625, 0.390625, 0.4609375, 0.4765625, 0.4453125, 0.5546875, 0.53125, 0.515625, 0.4453125, 0.484375, 0.3984375, 0.4609375, 0.4375, 0.484375, 0.5390625, 0.453125, 0.4921875, 0.5703125, 0.5546875, 0.484375, 0.5234375, 0.546875, 0.53125, 0.5234375, 0.4765625, 0.4765625, 0.3828125, 0.453125, 0.4921875, 0.5, 0.5078125, 0.4765625, 0.5703125, 0.4921875, 0.4453125, 0.4